### LLaMA로 라벨 정제하기

In [34]:
import json
import pandas as pd

with open('key_maps.json', 'r', encoding='utf-8') as f:
    key_maps = json.load(f)

train = pd.read_csv('train-no_special-no_hanja-polluted_lv.csv').drop(columns=['Unnamed: 0'])
print(key_maps)
train[train['polluted_lv']<0.2].sample(5)

{'0': '문화', '1': '스포츠', '2': '정치', '3': '사회', '4': '기술', '5': '경제', '6': '국제'}


,ID,text,target,polluted_lv,no_hanja
43,ynat-v1_train_00043,한국민족의 뿌리가 북방…5천년 전 한반도서 자생적으로 형성,0,0.032258,한국민족의 뿌리가 북방5천년 전 한반도서 자생적으로 형성
1720,ynat-v1_train_01720,페이스북 메신저 암호화 기능…저커버그 미래는 프라이빗,4,0.000000,페이스북 메신저 암호화 기능저커버그 미래는 프라이빗
536,ynat-v1_train_00536,제1차 세계대전 종전은 새로운 폭력의 시작이었다,2,0.038462,제1차 세계대전 종전은 새로운 폭력의 시작이었다
1719,ynat-v1_train_01719,北 대남테러 예상유형은…사이버 공격·요인 테러 가능성,2,0.000000,북 대남테러 예상유형은사이버 공격요인 테러 가능성
2798,ynat-v1_train_02798,인터뷰 류현진 친구에게 안타 맞는 것 싫어해…승부는 냉정,1,0.000000,인터뷰 류현진 친구에게 안타 맞는 것 싫어해승부는 냉정


In [35]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = 'Bllossom/llama-3.2-Korean-Bllossom-3B' # 'beomi/Llama-3-Open-Ko-8B'

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map='auto'
)

model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
      )
    )
    (norm): L

In [ ]:
from tqdm.notebook import tqdm

keys = list(key_maps.values())
keys_str = []
for idx, key in list(key_maps.items()):
    keys_str.append(f'{idx}: {key}')
keys_str = ', '.join(keys_str)

PROMPT = f'''기사 제목을 보고 주어진 분야 중 올바른 분야의 번호를 선택하세요.
- 기사 분야: {keys_str}
- 위 분야 중 올바른 분야의 번호를 선택하세요.
- 문맥적으로 올바르지 않은 데이터는 '불가'라고 출력하세요.
- 반드시 주어진 키워드 중 한 개만 선택하세요.'''

fewshot = [
    {"role": "user", "content": "구미 영천 등 경북 9개 시군 폭염주의보"},
    {"role": "assistant", "content" : "0"},
    {"role": "user", "content": "울산 진보3당 북구시설관리공단 설립 조례안 철회하라"},
    {"role": "assistant", "content": "2"},
    {"role": "user", "content": "국제유가곡물가올해 농업경영비 33 감소 전망"},
    {"role": "assistant", "content": "3"}
]

terminators = [
    tokenizer.convert_tokens_to_ids("<|end_of_text|>"),
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

# train['new_target'] = None
cnt = 0
for data in tqdm(train.iterrows(), desc='labeling', total=len(train)):
    data = data[1]
    
    if data['polluted_lv'] < 0.2:
        messages = [{"role": "system", "content": PROMPT}] + fewshot + [{"role": "user", "content": data['no_hanja']}]
        
        input_ids = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            return_tensors='pt'
        ).to(model.device)

        outputs = model.generate(
            input_ids,
            max_new_tokens=1024,
            eos_token_id=terminators,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=True,
            temperature=0.6,
            top_p=0.9
        )

        result = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)
        if result != '불가':
            cnt += 1
            train.loc[train['ID']==data['ID'], 'target'] = int(result)
print(cnt)
train.head()

labeling:   0%|          | 0/2800 [00:00<?, ?it/s]

In [44]:
train.to_csv('train-relabeled.csv')